In [18]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#allows to print the dataframe nicely
from IPython.core import display as ICD

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# import additional packages to insepct data
import pandas as pd
import os 
import random 
from zipfile import ZipFile
import datetime

In [20]:
# import helping functions
#from proj1_helpers import load_csv_data

In [21]:
# import files
data_folder='./data/'
zip_file = ZipFile(data_folder+'all.zip')
# zip file creates a list of files with certain properties
zip_file.infolist()

[<ZipInfo filename='train.csv' compress_type=deflate file_size=48103379 compress_size=17670847>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=109335584 compress_size=40089137>,
 <ZipInfo filename='sample-submission.csv' compress_type=deflate file_size=5682394 compress_size=1263762>]

Training set, Validation set and Test set.

In [22]:
# now we want to access the 'filename' property in the zipfile variable
# and we create a dictionary of dataframe
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df_train=dfs['train.csv']
df_test=dfs['test.csv']
df_sample_submission=dfs['sample-submission.csv']

variables prefixed with PRI (for PRImitives) are “raw” quantities about the bunch collision as measured by the detector.
variables prefixed with DER (for DERived) are quantities computed from the primitive features, which were selected by  the physicists of ATLAS
it can happen that for some entries some variables are meaningless or cannot be computed; in this case, their value is −999.0, which is outside the normal range of all variables

In [76]:
df_train_independent=df_train[['DER_mass_MMC','DER_mass_transverse_met_lep', 'DER_mass_vis', 'DER_pt_h', 'DER_deltar_tau_lep', 
                              'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality', 'PRI_tau_pt', 
                              'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta','PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 
                               'PRI_met_sumet','PRI_jet_num', 'PRI_jet_all_pt' ]]
ICD.display(df_train.columns.values)
df_train_dependent=df_train[['DER_deltaeta_jet_jet','DER_mass_jet_jet', 'DER_prodeta_jet_jet','DER_lep_eta_centrality',
                             'PRI_jet_leading_pt', 'PRI_jet_leading_eta','PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
                             'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi','PRI_jet_num']]
df_train_independent

array(['Id', 'Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet',
       'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_deltar_tau_lep',
       'DER_pt_tot', 'DER_sum_pt', 'DER_pt_ratio_lep_tau',
       'DER_met_phi_centrality', 'DER_lep_eta_centrality', 'PRI_tau_pt',
       'PRI_tau_eta', 'PRI_tau_phi', 'PRI_lep_pt', 'PRI_lep_eta',
       'PRI_lep_phi', 'PRI_met', 'PRI_met_phi', 'PRI_met_sumet',
       'PRI_jet_num', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta',
       'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi',
       'PRI_jet_all_pt'], dtype=object)

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_all_pt
0,138.470,51.655,97.827,27.980,3.064,41.928,197.760,1.582,1.396,32.638,1.017,0.381,51.626,2.273,-2.414,16.824,-0.277,258.733,2,113.497
1,160.937,68.768,103.235,48.146,3.473,2.078,125.157,0.879,1.414,42.014,2.039,-3.011,36.918,0.501,0.103,44.704,-1.916,164.546,1,46.226
2,-999.000,162.172,125.953,35.635,3.148,9.336,197.814,3.776,1.414,32.154,-0.705,-2.093,121.409,-0.953,1.052,54.283,-2.186,260.414,1,44.251
3,143.905,81.417,80.943,0.414,3.310,0.414,75.968,2.354,-1.285,22.647,-1.655,0.010,53.321,-0.522,-3.100,31.082,0.060,86.062,0,0.000
4,175.864,16.915,134.805,16.405,3.891,16.405,57.983,1.056,-1.385,28.209,-2.197,-2.231,29.774,0.798,1.569,2.723,-0.871,53.131,0,0.000
5,89.744,13.550,59.149,116.344,1.362,61.619,278.876,0.588,0.479,53.651,0.371,1.329,31.565,-0.884,1.857,40.735,2.237,282.849,3,193.660
6,148.754,28.862,107.782,106.130,2.941,2.545,305.967,3.371,1.393,28.850,1.113,2.409,97.240,0.675,-0.966,38.421,-1.443,294.074,2,179.877
7,154.916,10.418,94.714,29.169,2.897,1.526,138.178,0.365,-1.305,78.800,0.654,1.547,28.740,0.506,-1.347,22.275,-1.761,187.299,1,30.638
8,105.594,50.559,100.989,4.288,2.904,4.288,65.333,0.675,-1.366,39.008,2.433,-2.532,26.325,0.210,1.884,37.791,0.024,129.804,0,0.000
9,128.053,88.941,69.272,193.392,1.609,28.859,255.123,0.599,0.538,54.646,-1.533,0.416,32.742,-0.317,-0.636,132.678,0.845,294.741,1,167.735


# Test solving

In [86]:
df_train_independent['PRI_jet_num']=df_train_independent['PRI_jet_num'].astype('float')
yb, input_data, ids=np.array(df_train['Prediction']), np.array(df_train_independent), np.array(df_train['Id'])


C:\Users\Tullio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [79]:
ICD.display(input_data)


array([[ 138.47 ,   51.655,   97.827, ...,  258.733,    2.   ,  113.497],
       [ 160.937,   68.768,  103.235, ...,  164.546,    1.   ,   46.226],
       [-999.   ,  162.172,  125.953, ...,  260.414,    1.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ...,  198.907,    1.   ,   41.992],
       [  94.951,   19.362,   68.812, ...,  112.718,    0.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ...,   99.405,    0.   ,    0.   ]])

In [91]:
# transform yb into numerical values
yb[np.where(yb=='b')] = -1.
yb[np.where(yb=='s')] = 1.
yb=yb.astype('float')

#Cut dataframe for fast testing
lines_cut=200000
tx=input_data[:lines_cut]
y=yb[:lines_cut]
y.shape
#input_data.shape

C:\Users\Tullio\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
C:\Users\Tullio\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


(200000,)

In [81]:
#Import functions
from proj1_helpers import *
from implementations import *

## Stochastic gradient descent 

In [89]:
# Define the parameters of the algorithm.
max_iters = 30
gamma = 0.1
batch_size = 2000

# Initialization
w_initial = np.zeros(tx.shape[1])

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = stochastic_gradient_descent(y, tx, w_initial, batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

SGD(0/29): loss=18011148.84390846, wavg=12.7038445, w1=-3.256724
SGD(1/29): loss=6102124376216975.0, wavg=-246818.28503168063, w1=22650.819260423992
SGD(2/29): loss=2.1630737935533594e+24, wavg=4712822502.36952, w1=-355202282.19604224
SGD(3/29): loss=7.77193460557274e+32, wavg=-89787174033371.94, w1=6109087747859.03
SGD(4/29): loss=2.8037207964563767e+41, wavg=1.7088397631991498e+18, w1=-1.0958105518600872e+17
SGD(5/29): loss=1.012640273498446e+50, wavg=-3.2504959883406465e+22, w1=2.015702860127829e+21
SGD(6/29): loss=3.658699577118203e+58, wavg=6.1811447403703715e+26, w1=-3.762392358084615e+25
SGD(7/29): loss=1.3220339946750275e+67, wavg=-1.1752178970049946e+31, w1=7.080709616859469e+29
SGD(8/29): loss=4.777178709353393e+75, wavg=2.2342414597474434e+35, w1=-1.338652887007585e+34
SGD(9/29): loss=1.7262518711748951e+84, wavg=-4.247382465779522e+39, w1=2.5371332512603333e+38
SGD(10/29): loss=6.237893462293156e+92, wavg=8.07423832454677e+43, w1=-4.8151450208158235e+42
SGD(11/29): loss=2.2

## Least Squares

In [92]:
w=least_squares(y,tx)
loss=compute_mse(y, tx, w)
print('Weigths: ',w,"\n\n",'Loss: ',loss)

Weigths:  [ 1.92057425e-04 -7.81456044e-03 -3.36943262e-03 -1.16036286e-03
  9.13830175e-02 -1.66544657e-03 -2.20070501e+00 -2.95369697e-01
  1.18915490e-01  2.20476711e+00 -2.02554738e-03 -1.23484203e-03
  2.21320919e+00  1.33097776e-03  5.73974626e-05  3.38886580e-03
 -5.46660038e-04 -1.10462246e-03  1.75714065e-02  2.20107262e+00] 

 Loss:  0.36042244524389344


# Other stuff
Build Polynomial

In [50]:
# from helpers import polynomial_regression
from helpers import build_poly
degree=2
tx=build_poly(input_data[:3,:3], degree)
#you can select the feature space by slicing the array
ICD.display(tx[[0,1,2]])

#trial of slicing without for loop
# ICD.display(np.transpose(tx[[0,1,2]],(0,2,1)))
a = np.tensordot(np.transpose(tx[[0,1]],(0,2,1)),(tx[[0,1]]),axes=([2],[2]))
b = tx[[0,1,2]].T.dot(yb[[0,1,2]])

# ICD.display(tx.shape)
np.transpose(tx,(0,2,1)).shape
ICD. display(a)

array([[[ 1.00000000e+00,  1.38470000e+02,  1.91739409e+04],
        [ 1.00000000e+00,  1.60937000e+02,  2.59007180e+04],
        [ 1.00000000e+00, -9.99000000e+02,  9.98001000e+05]],

       [[ 1.00000000e+00,  5.16550000e+01,  2.66823903e+03],
        [ 1.00000000e+00,  6.87680000e+01,  4.72903782e+03],
        [ 1.00000000e+00,  1.62172000e+02,  2.62997576e+04]],

       [[ 1.00000000e+00,  9.78270000e+01,  9.57012193e+03],
        [ 1.00000000e+00,  1.03235000e+02,  1.06574652e+04],
        [ 1.00000000e+00,  1.25953000e+02,  1.58641582e+04]]])

array([[[[ 1.93134109e+04,  2.60626550e+04,  9.97003000e+05],
         [ 2.72089403e+03,  4.79880582e+03,  2.64629296e+04]],

        [[-1.91323435e+07, -2.58487781e+07, -9.97163637e+08],
         [-2.65711912e+06, -4.71310300e+06, -2.62472199e+07]],

        [[ 1.91392178e+10,  2.58531300e+10,  9.95980140e+11],
         [ 2.66426229e+09,  4.72138479e+09,  2.62514039e+10]]],


       [[[ 1.93134109e+04,  2.60626550e+04,  9.97003000e+05],
         [ 2.72089403e+03,  4.79880582e+03,  2.64629296e+04]],

        [[ 3.11905030e+06,  4.21149021e+06,  1.61779171e+08],
         [ 4.36317525e+05,  7.71698215e+05,  4.27628819e+06]],

        [[ 5.04927496e+08,  6.81946349e+08,  2.62424627e+10],
         [ 7.04209862e+07,  1.24700423e+08,  6.92446835e+08]]]])

In [73]:
from helpers import ridge_regression
lambda_=10
aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])

tx_t=np.transpose(tx,(0,2,1))
ICD.display(tx_t.shape)
ICD.display(tx.shape)

a = np.tensordot(tx_t, tx[0], axes=1)+ aI


b = tx.T.dot(yb[:3])
ICD.display(tx)
ICD.display(tx_t)
ICD.display(a)
# np.linalg.solve(a, b)

(3, 3, 3)

(3, 3, 3)

array([[[ 1.00000000e+00,  1.38470000e+02,  1.91739409e+04],
        [ 1.00000000e+00,  1.60937000e+02,  2.59007180e+04],
        [ 1.00000000e+00, -9.99000000e+02,  9.98001000e+05]],

       [[ 1.00000000e+00,  5.16550000e+01,  2.66823903e+03],
        [ 1.00000000e+00,  6.87680000e+01,  4.72903782e+03],
        [ 1.00000000e+00,  1.62172000e+02,  2.62997576e+04]],

       [[ 1.00000000e+00,  9.78270000e+01,  9.57012193e+03],
        [ 1.00000000e+00,  1.03235000e+02,  1.06574652e+04],
        [ 1.00000000e+00,  1.25953000e+02,  1.58641582e+04]]])

array([[[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 1.38470000e+02,  1.60937000e+02, -9.99000000e+02],
        [ 1.91739409e+04,  2.59007180e+04,  9.98001000e+05]],

       [[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 5.16550000e+01,  6.87680000e+01,  1.62172000e+02],
        [ 2.66823903e+03,  4.72903782e+03,  2.62997576e+04]],

       [[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
        [ 9.78270000e+01,  1.03235000e+02,  1.25953000e+02],
        [ 9.57012193e+03,  1.06574652e+04,  1.58641582e+04]]])

array([[[ 6.30000000e+01, -6.99593000e+02,  1.04307566e+06],
        [-6.99593000e+02,  1.04313566e+06, -9.90179600e+08],
        [ 1.04307566e+06, -9.90179600e+08,  9.97044483e+11]],

       [[ 6.30000000e+01, -6.99593000e+02,  1.04307566e+06],
        [ 2.82595000e+02, -1.43729845e+05,  1.64619389e+08],
        [ 3.36970344e+04, -2.51429096e+07,  2.64208306e+10]],

       [[ 6.30000000e+01, -6.99593000e+02,  1.04307566e+06],
        [ 3.27015000e+02, -9.56066111e+04,  1.30250810e+08],
        [ 3.60917454e+04, -1.28079388e+07,  1.62919788e+10]]])

In [105]:
#function with foor loop
def ridge_regression_loop(y, tx, degrees, lambda_):
    """calculate the least squares."""
    #weights has dimension: number of degrees, number of features
    
    weights=np.ones((degrees+1,tx.shape[0]))
    
    tx_t=np.transpose(tx,(0,2,1))
    
    # going through every feature
    for i in range(tx.shape[0]):
        
        aI = 2 * tx[i].shape[0] * lambda_ * np.identity(tx[i].shape[1])
        print (aI)
        print (tx_t[i])
        print (tx[i])
        print (y)
        a = tx_t[i].dot(tx[i])
        print ('a')
        print (a)
        b = tx_t[i].dot(y)
        print ('b')
        print (b)
        
        w=np.linalg.solve(a, b)
        print (w)
        weights[:][i]=w
    
    return weights

In [104]:
#testing function least square loop

lambda_=0
degrees=2
tx=build_poly(input_data[:3,:3], degree)
y=yb[:3]
ICD.display(tx)
ICD.display(y)
ridge_regression_loop(y, tx, degrees, lambda_)


array([[[ 1.00000000e+00,  1.38470000e+02,  1.91739409e+04],
        [ 1.00000000e+00,  1.60937000e+02,  2.59007180e+04],
        [ 1.00000000e+00, -9.99000000e+02,  9.98001000e+05]],

       [[ 1.00000000e+00,  5.16550000e+01,  2.66823903e+03],
        [ 1.00000000e+00,  6.87680000e+01,  4.72903782e+03],
        [ 1.00000000e+00,  1.62172000e+02,  2.62997576e+04]],

       [[ 1.00000000e+00,  9.78270000e+01,  9.57012193e+03],
        [ 1.00000000e+00,  1.03235000e+02,  1.06574652e+04],
        [ 1.00000000e+00,  1.25953000e+02,  1.58641582e+04]]])

array([1, -1, -1], dtype=object)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 1.38470000e+02  1.60937000e+02 -9.99000000e+02]
 [ 1.91739409e+04  2.59007180e+04  9.98001000e+05]]
[[ 1.00000000e+00  1.38470000e+02  1.91739409e+04]
 [ 1.00000000e+00  1.60937000e+02  2.59007180e+04]
 [ 1.00000000e+00 -9.99000000e+02  9.98001000e+05]]
[1 -1 -1]
a
[[ 3.00000000e+00 -6.99593000e+02  1.04307566e+06]
 [-6.99593000e+02  1.04307566e+06 -9.90179600e+08]
 [ 1.04307566e+06 -9.90179600e+08  9.97044483e+11]]
b
[-1.0 976.533 -1004727.7770690001]


TypeError: No loop matching the specified signature and casting
was found for ufunc solve1

In [65]:
# understanding tensordot
A = np.random.randint(2, size=(2, 3, 5))
B = np.random.randint(2, size=(3, 2, 4))
ICD.display(A)
ICD.display(B)

np.tensordot(A, B, axes=([0,1],[1,0]))

array([[[0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]],

       [[1, 0, 0, 1, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0]]])

array([[[0, 0, 0, 1],
        [0, 1, 1, 1]],

       [[1, 1, 1, 0],
        [1, 0, 1, 1]],

       [[0, 0, 1, 1],
        [0, 1, 0, 1]]])

array([[1, 2, 3, 4],
       [1, 1, 1, 2],
       [0, 1, 0, 2],
       [0, 2, 1, 2],
       [0, 0, 0, 0]])